In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Fri_Jan__6_16:45:21_PST_2023
Cuda compilation tools, release 12.0, V12.0.140
Build cuda_12.0.r12.0/compiler.32267302_0


In [2]:
!python3

Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
KeyboardInterrupt
>>> 

In [3]:
!nvidia-smi

Tue Aug 27 22:01:14 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1060        Off |   00000000:01:00.0  On |                  N/A |
| N/A   53C    P8              5W /   78W |      56MiB /   6144MiB |     28%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Api

In [4]:
import time
import numpy as np
import os
import _pickle as cPickle
import numpy as np
import imagenet_stubs
from imagenet_stubs.imagenet_2012_labels import label_to_name as IMAGENET_CLASS_NAMES
import pycuda.autoinit
import pycuda.driver as drv
import numpy
from PIL import Image
import imagehash

from pycuda.compiler import SourceModule

PATH = "../data/cifar-10-batches-py/"
IMAGENET_PATH = "../data/Imagenet32_train/"

In [16]:
def unpickleImages(pairs):
    images = set()
    for elem in pairs:
        images.add(elem[0])
        images.add(elem[1])

    return list(images)

In [17]:
unpickleImages([(1,2, 0.85),(3,4,0.34)])

[1, 2, 3, 4]

In [21]:
nnLSH = cuNN(8, 8)
nnLSH.compute_hash(CIFAR_IMAGES)
pairs = nnLSH.compute_candidate()
duplicates = nnLSH.check_candidate(pairs)

# get images from duplicates
DUPLICATED_IMAGES = unpickleImages(duplicates)

perceptualLSH = cuPerceptual(8, 8)
perceptualLSH.compute_hash(DUPLICATED_IMAGES)
pairs2 = perceptualLSH.compute_candidate()
duplicates2 = perceptualLSH.check_candidate(pairs2)

#calculate metrics

NameError: name 'cuNN' is not defined

In [22]:
perceptualLSH = cuPerceptual(8, 8)
perceptualLSH.compute_hash(CIFAR_IMAGES)
pairs3 = perceptualLSH.compute_candidate()
duplicates3 = perceptualLSH.check_candidate(pairs3)

# get images from duplicates
DUPLICATED_IMAGES2 = unpickleImages(duplicates3)

nnLSH = cuNN(8, 8)
nnLSH.compute_hash(DUPLICATED_IMAGES2)
pairs4 = nnLSH.compute_candidate()
duplicates4 = nnLSH.check_candidate(pairs4)

#calculate metrics

NameError: name 'cuPerceptual' is not defined